* This notebook uses data from https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/.
* It contains Experiment with LightGBM

In [8]:
import pandas as pd
import numpy as np

In [10]:
train_data = pd.read_csv('data/dengue_features_train.csv')
#train_data['week_start_date'] = pd.to_datetime(train_data.week_start_date)
X_train = train_data.drop('week_start_date',axis=1)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   city                                   1456 non-null   object 
 1   year                                   1456 non-null   int64  
 2   weekofyear                             1456 non-null   int64  
 3   ndvi_ne                                1262 non-null   float64
 4   ndvi_nw                                1404 non-null   float64
 5   ndvi_se                                1434 non-null   float64
 6   ndvi_sw                                1434 non-null   float64
 7   precipitation_amt_mm                   1443 non-null   float64
 8   reanalysis_air_temp_k                  1446 non-null   float64
 9   reanalysis_avg_temp_k                  1446 non-null   float64
 10  reanalysis_dew_point_temp_k            1446 non-null   float64
 11  rean

In [11]:
labels = pd.read_csv('data/dengue_labels_train.csv')
labels.head()
merged = X_train.merge(labels,on=['city','year','weekofyear'])
y_train = merged['total_cases']
X_train = merged.drop('total_cases',axis=1)
X_train.head()

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [12]:
X_train['year'] = X_train['year'].astype('object')
X_train['weekofyear'] = X_train['weekofyear'].astype('object')

X_train['week_year'] = X_train['city']+"_"+X_train['weekofyear'].astype('str')+"_"+X_train['year'].astype('str')
X_train = X_train.drop(['city','year','weekofyear'],axis=1)
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456 entries, 0 to 1455
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ndvi_ne                                1262 non-null   float64
 1   ndvi_nw                                1404 non-null   float64
 2   ndvi_se                                1434 non-null   float64
 3   ndvi_sw                                1434 non-null   float64
 4   precipitation_amt_mm                   1443 non-null   float64
 5   reanalysis_air_temp_k                  1446 non-null   float64
 6   reanalysis_avg_temp_k                  1446 non-null   float64
 7   reanalysis_dew_point_temp_k            1446 non-null   float64
 8   reanalysis_max_air_temp_k              1446 non-null   float64
 9   reanalysis_min_air_temp_k              1446 non-null   float64
 10  reanalysis_precip_amt_kg_per_m2        1446 non-null   float64
 11  rean

In [18]:
num_col = [col for col in X_train.columns if X_train[col].dtype=='int' or X_train[col].dtype=='float64']
cat_col = [col for col in X_train.columns if X_train[col].dtype=='object']


In [29]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

num_processor = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('normalizer',Normalizer())
])
cat_processor = Pipeline(steps=[
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num',num_processor,num_col),
    ('cat',cat_processor,cat_col)
])

x = preprocessor.fit_transform(X_train)

In [32]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
model = XGBRegressor(n_estimators=5000,learning_rate=0.01)
pipeline = Pipeline(steps=[
    ('processor',preprocessor),
    ('model',model)
])

score = -1*cross_val_score(pipeline,X_train,y_train,cv=3,scoring='neg_mean_absolute_error')
print(f'MSE Loss mean = {np.mean(score)} max = {np.max(score)}')

MSE Loss mean = 26.103497811381786 max = 35.55268544253008


,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [37]:
model = pipeline.fit(X_train,y_train)

In [50]:
X_test = pd.read_csv('data/dengue_features_test.csv')
X_test.head()
X_test = X_test.drop('week_start_date',axis=1)
X_test['year'] = X_test['year'].astype('object')
X_test['weekofyear'] = X_test['weekofyear'].astype('object')

X_test['week_year'] = X_test['city']+"_"+X_test['weekofyear'].astype('str')+"_"+X_test['year'].astype('str')
X_test = X_test.drop(['city','year','weekofyear'],axis=1)

In [51]:
X_test['total_cases'] = model.predict(X_test)

In [52]:
X_test.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,...,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,week_year,total_cases
0,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,294.527143,301.1,296.4,...,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2,sj_18_2008,14.678601
1,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,294.395714,300.8,296.7,...,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3,sj_19_2008,11.341932
2,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,299.357143,295.308571,302.2,296.4,...,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0,sj_20_2008,10.495125
3,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,294.402857,303.0,296.9,...,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3,sj_21_2008,37.485123
4,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,294.760000,302.3,297.3,...,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1,sj_22_2008,32.415176


In [53]:
submission = pd.DataFrame(X_test.week_year.str.split('_').tolist(),
                                 columns = ['city','weekofyear','year'])

In [56]:
submission['total_cases'] = X_test['total_cases'].astype('int64')
submission = submission[['city','year','weekofyear','total_cases']]
submission

,city,year,weekofyear,total_cases
0,sj,2008,18,14
1,sj,2008,19,11
2,sj,2008,20,10
3,sj,2008,21,37
4,sj,2008,22,32
...,...,...,...,...
411,iq,2013,22,4
412,iq,2013,23,5
413,iq,2013,24,4
414,iq,2013,25,3


In [57]:
submission.to_csv('submission.csv', index=False)